<a href="https://colab.research.google.com/github/babymlin/Cifar10_80_percent/blob/main/Cifar10_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow-addons
import tensorflow as tf
import tensorflow_addons as tfa
import matplotlib.pyplot as plt

     |████████████████████████████████| 686kB 7.6MB/s 


In [2]:
from tensorflow.keras.datasets.cifar10 import load_data
(X_train, y_train), (X_test, y_test) = load_data()

170500096/170498071 [==============================] - 4s 0us/step


In [27]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(50000, 32, 32, 3)
(50000, 1)
(10000, 32, 32, 3)
(10000, 1)


In [28]:
X_train_norm = X_train / 255.0
X_test_norm = X_test / 255.0

In [35]:
class_names = [
  "airplane",										
  "automobile",										
  "bird",										
  "cat",										
  "deer",										
  "dog",										
  "frog",										
  "horse",										
  "ship",										
  "truck"
]

In [36]:
input_shape = (X_train.shape[1], X_train.shape[2], X_train.shape[3])
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Softmax, MaxPooling2D, Flatten, BatchNormalization, ReLU, GlobalAveragePooling2D, Dropout
layers1 = [
  Conv2D(filters=64, kernel_size=3, strides=1, padding="same", activation="relu", input_shape=input_shape),
  MaxPooling2D(),
  Conv2D(filters=128, kernel_size=3, strides=1, padding="same", activation="relu"),
  MaxPooling2D(),
  Conv2D(filters=256, kernel_size=3, strides=1, padding="same", activation="relu"),
  MaxPooling2D(),
  GlobalAveragePooling2D(),
  Dense(units=128, activation="relu"),
  Dropout(0.25),
  Dense(units=10),
  Softmax()
] 
layers = [
  Conv2D(filters=64, kernel_size=3, strides=1, padding="same", input_shape=input_shape),
  BatchNormalization(),
  ReLU(),
  Conv2D(filters=64, kernel_size=3, strides=2, padding="same"),
  BatchNormalization(),
  ReLU(),
  Conv2D(filters=128, kernel_size=3, strides=1, padding="same", activation="relu"),
  BatchNormalization(),
  ReLU(),
  Conv2D(filters=128, kernel_size=3, strides=2, padding="same"),
  BatchNormalization(),
  ReLU(),
  Conv2D(filters=256, kernel_size=3, strides=1, padding="same"),
  BatchNormalization(),
  ReLU(),
  Conv2D(filters=256, kernel_size=3, strides=2, padding="same"),
  BatchNormalization(),
  ReLU(),
  GlobalAveragePooling2D(),
  Dense(units=128, activation="relu"),
  Dropout(0.25),
  Dense(units=10),
  Softmax()
] 
model = Sequential(layers1)
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_27 (Conv2D)           (None, 32, 32, 64)        1792      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 16, 16, 128)       73856     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 8, 8, 128)         0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 8, 8, 256)         295168    
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 4, 4, 256)         0         
_________________________________________________________________
global_average_pooling2d_3 ( (None, 256)              

In [37]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger
callbacks = [
  EarlyStopping(monitor="val_loss",
        restore_best_weights=True,
        patience=5,
        ),
  ModelCheckpoint("Cifar10v2-BestModel.h5", monitor="val_accuracy", save_best_only=True),
  CSVLogger("Cifar10v2.csv")
]

In [38]:
radam = tfa.optimizers.RectifiedAdam()
ranger = tfa.optimizers.Lookahead(radam)
model.compile(optimizer=ranger, loss="SparseCategoricalCrossentropy", metrics=["accuracy"])

In [39]:
batch_size = 64
epochs = 100
model.fit(X_train_norm,
    y_train,
    batch_size=batch_size, 
    epochs=epochs, 
    validation_split=0.1, 
    callbacks=callbacks,
    verbose=2
)

Epoch 1/100
704/704 - 7s - loss: 1.9568 - accuracy: 0.2590 - val_loss: 1.9064 - val_accuracy: 0.2898
Epoch 2/100
704/704 - 5s - loss: 1.6478 - accuracy: 0.3893 - val_loss: 1.5200 - val_accuracy: 0.4378
Epoch 3/100
704/704 - 5s - loss: 1.4803 - accuracy: 0.4559 - val_loss: 1.3500 - val_accuracy: 0.5092
Epoch 4/100
704/704 - 5s - loss: 1.3567 - accuracy: 0.5059 - val_loss: 1.3378 - val_accuracy: 0.5144
Epoch 5/100
704/704 - 5s - loss: 1.2590 - accuracy: 0.5484 - val_loss: 1.2394 - val_accuracy: 0.5384
Epoch 6/100
704/704 - 5s - loss: 1.1779 - accuracy: 0.5764 - val_loss: 1.0606 - val_accuracy: 0.6336
Epoch 7/100
704/704 - 5s - loss: 1.1078 - accuracy: 0.6058 - val_loss: 1.0859 - val_accuracy: 0.6142
Epoch 8/100
704/704 - 5s - loss: 1.0488 - accuracy: 0.6270 - val_loss: 1.0053 - val_accuracy: 0.6446
Epoch 9/100
704/704 - 5s - loss: 0.9954 - accuracy: 0.6472 - val_loss: 0.9219 - val_accuracy: 0.6778
Epoch 10/100
704/704 - 5s - loss: 0.9427 - accuracy: 0.6663 - val_loss: 0.8970 - val_accura

In [40]:
model.evaluate(X_test_norm, y_test)

313/313 [==============================] - 1s 3ms/step - loss: 0.6508 - accuracy: 0.7805


[0.6508117914199829, 0.7804999947547913]